In [1]:
import requests
import re
import json
import pandas as pd
import time
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

In [2]:
# 1. Obtener el HTML de la página del ranking
load_dotenv()
url = os.getenv("TARGET_URL_CPI")
BASE_URL = os.getenv("BASE_URL")

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.raise_for_status()  # Lanza error si la petición falla

In [3]:
# 2. Parsear HTML con BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

In [4]:
# 3. Buscar la tabla principal y extraer los links
table = soup.find("table")
country_links = []


if table:
    rows = table.find_all("tr")
    for row in rows:
        cell = row.find("td")
        if cell and cell.a:
            relative_link = cell.a["href"]
            full_url = f"{BASE_URL}{relative_link}"
            country_links.append(full_url)

# Convertir la lista en DataFrame
df_links = pd.DataFrame(country_links, columns=["url"])

# Guardar como archivo Parquet
df_links.to_parquet("data/country_cpi_links.parquet", index=False)

In [5]:
# 4. Mostrar resultado
print("Links encontrados:", df_links.size)

Links encontrados: 182


In [6]:
# configuracion necesaria para descargar datos

master_df = pd.DataFrame()

pattern = r'arrayToDataTable\((\[\[.*?\]\])\)'

sleep_duration = 5

In [7]:
# Cargar los países válidos desde el archivo parquet
df_countries = pd.read_parquet("data/countries_with_hpi.parquet")
valid_countries = set(df_countries["country"].unique())

In [ ]:
start_time = time.time()
for url in country_links:
    # Extraer país desde el URL
    country = urlparse(url).path.strip("/").split("/")[0]

    # Verificar si el país está en la lista válida
    if country not in valid_countries:
        print(f"⏭️ País {country} no está en la lista, se omite.")
        continue

    try:
        response = requests.get(url, headers=headers)
        html = response.text

        match = re.search(pattern, html, re.DOTALL)
        if match:
            data_str = match.group(1)
            data = json.loads(data_str)

            df = pd.DataFrame(data[1:], columns=data[0])
            df.columns = ["Period", "Value"]
            df["Country"] = country

            df_pivot = df.pivot(index="Period", columns="Country", values="Value")
            master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

            print(f"✅ Datos extraídos correctamente para {country}")
        else:
            print(f"⚠️ No se encontraron datos en {url}")
    except Exception as e:
        print(f"❌ Error procesando {url}: {e}")
    
    time.sleep(sleep_duration)

end_time = time.time()
total_sleep_time = len(valid_countries) * sleep_duration
effective_time = end_time - start_time - total_sleep_time

print(f"⏳ Tiempo total (incluyendo esperas): {end_time - start_time:.2f} s")
print(f"😴 Tiempo en espera: {total_sleep_time:.2f} s")
print(f"⚡ Tiempo efectivo de ejecución: {effective_time:.2f} s")

In [10]:
# obtain paraguay url
template_url = country_links[0]
country = "Paraguay"

# Separar por "/"
parts = template_url.strip("/").split("/")

# Reemplazar el país (posición 3: después del dominio)

parts[3] = country

# Reconstruir la URL
custom_url = "/".join(parts) + "/"

In [11]:
# extract and include paraguay

response = requests.get(custom_url, headers=headers)
html = response.text

match = re.search(pattern, html, re.DOTALL)
if match:
    data_str = match.group(1)
    data = json.loads(data_str)

    df = pd.DataFrame(data[1:], columns=data[0])
    df.columns = ["Period", "Value"]
    df["Country"] = country

    df_pivot = df.pivot(index="Period", columns="Country", values="Value")
    master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

    print(f"✅ Datos extraídos correctamente para {country}")

✅ Datos extraídos correctamente para Paraguay


In [16]:
# Resetear el índice para separar "Period"
master_df = master_df.reset_index()

# Mapeo de mes a trimestre
month_to_quarter = {
    "Jan": "Q1", "Feb": "Q1", "Mar": "Q1",
    "Apr": "Q2", "May": "Q2", "Jun": "Q2",
    "Jul": "Q3", "Aug": "Q3", "Sep": "Q3",
    "Oct": "Q4", "Nov": "Q4", "Dec": "Q4"
}

# Extraer mes y año desde "Period"
master_df[["Month", "Year"]] = master_df["Period"].str.extract(r"(\w{3})\s+(\d{4})")

# Filtrar filas válidas (donde se extrajo correctamente el año)
master_df = master_df[master_df["Year"].notna()].copy()

# Mapear trimestre y convertir año
master_df["Quarter"] = master_df["Month"].map(month_to_quarter)
master_df["Year"] = master_df["Year"].astype(int)

# Reordenar las columnas: primero Quarter y Year
cols = ["Quarter", "Year"] + [col for col in master_df.columns if col not in ["Period", "Month", "Quarter", "Year"]]
master_df = master_df[cols]

# Ordenar por Year y Quarter
master_df = master_df.sort_values(by=["Year", "Quarter"]).reset_index(drop=True)

In [17]:
master_df.head()

,Quarter,Year,level_0,index,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
0,Q1,1960,196,196,NaN,NaN,NaN,NaN,NaN,15.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.41,NaN,NaN
1,Q1,1960,262,262,NaN,NaN,NaN,NaN,NaN,15.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.37,NaN,NaN
2,Q1,1960,460,460,NaN,NaN,NaN,NaN,NaN,15.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.41,NaN,NaN
3,Q2,1960,0,0,NaN,NaN,NaN,NaN,NaN,15.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.54,NaN,NaN
4,Q2,1960,526,526,NaN,NaN,NaN,NaN,NaN,15.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.57,NaN,NaN


In [18]:
master_df.tail()

,Quarter,Year,level_0,index,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
782,Q1,2025,525,525,NaN,138.27,135.71,7245.38,142.53,163.5,...,101.6,116.29,125.66,131.19,106.86,NaN,1133.49,319.799,136.1,138.2
783,Q2,2025,65,65,NaN,138.64,134.79,7276.54,140.76,163.4,...,101.9,116.38,126.44,131.50,107.61,NaN,1167.42,320.795,137.7,NaN
784,Q2,2025,591,591,NaN,138.40,134.67,7295.46,140.84,NaN,...,102.1,116.27,126.40,131.96,107.40,NaN,1185.16,321.465,138.0,NaN
785,Q3,2025,393,393,NaN,138.81,134.22,NaN,NaN,NaN,...,NaN,116.52,126.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
786,NaN,2025,459,459,NaN,138.76,135.32,7312.97,141.44,NaN,...,102.4,116.31,127.27,132.71,107.66,NaN,1201.39,322.561,138.4,NaN


In [21]:
# Guardar como archivo Parquet
master_df.to_parquet("data/cpi_values.parquet", index=False)